## 네이버 영화평 감성 분석

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t')
test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t')

In [3]:
print(train_df.shape, test_df.shape)
train_df.head(3)

(150000, 3) (50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


### 1. Data Preprocessing

In [5]:
# Null 데이터가 있는지 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [6]:
# Null 데이터 제거
train_df.dropna(how='any', inplace=True) 
train_df.shape

(149995, 3)

In [7]:
# 중복여부 확인
train_df.document.nunique()

146182

In [8]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True) 
train_df.shape

(146182, 3)

In [9]:
# 긍정(1)/부정(0) 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- Test data set

In [11]:
print(test_df.isna().sum())
test_df.dropna(how='any', inplace=True)
print(test_df.document.nunique())
test_df.drop_duplicates(subset=['document'], inplace=True)
print(test_df.shape)
test_df.label.value_counts()

id          0
document    3
label       0
dtype: int64
49157
(49157, 3)


1    24711
0    24446
Name: label, dtype: int64

### 2. Text Preprocess

In [13]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [14]:
# '' 만 남은 데이터 --> np.nan 으로 대체한 후 제거
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [15]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

In [16]:
# Test data set
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

In [17]:
# Save
train_df.to_csv('./data/Naver_movie_train_preprocessed.tsv', sep='\t', index=False)
test_df.to_csv('./data/Naver_movie_test_prerocessed.tsv', sep='\t', index=False)

### 3. Korean Preprocess

In [19]:
from konlpy.tag import Okt
okt = Okt()

In [20]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [21]:
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [23]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']
' '.join([word for word in okt.morphs(text, stem=True) if word not in stopwords])

'교도소 이야기 구먼 솔직하다 재미 없다 평점 조정'

In [24]:
from tqdm.notebook import tqdm

X_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_train.append(tmp_str)

  0%|          | 0/145393 [00:00<?, ?it/s]

In [25]:
X_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_test.append(tmp_str)

  0%|          | 0/48852 [00:00<?, ?it/s]

In [26]:
y_train = train_df.label.values 
y_test = test_df.label.values

In [27]:
X_train

['아 더빙 진짜 짜증나다 목소리',
 '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다',
 '너 무재 밓었 다그 래서 보다 추천 다',
 '교도소 이야기 구먼 솔직하다 재미 없다 평점 조정',
 '사이 몬페 그 익살스럽다 연기 돋보이다 영화 스파이더맨 에서 늙다 보이다 커스틴 던스트 너무나도 이쁘다 보이다',
 '막 걸음 마 떼다 세 부터 초등학교 학년 생인 살다 영화 ㅋㅋㅋ 별 반개 아깝다 움',
 '원작 긴장감 제대로 살리다',
 '별 반개 아깝다 욕 나오다 이응경 길용우 연 기 생활 몇 년 인지 정말 발 로 해도 그것 보단 낫다 납치 감금 만 반복 반복 드라마 가족 없다 연기 못 사람 만 모 엿 네',
 '액션 없다 재미 있다 몇 안되다 영화',
 '왜케 평점 낮다 꽤 볼 만 데 헐리우드 식 화려하다 너무 길들이다 있다',
 '인피니트 짱 이다 진짜 짱 이다',
 '볼때 마다 눈물나다 죽다 년대 향수 자극 허진호 감성 절제 멜로 달인 이다',
 '울면 서 손 고 횡단보도 건너다 때 뛰다 치다 올 뻔 이범수 연기 드럽다 못',
 '담백하다 깔끔하다 좋다 신 문 기 사 로만 보다 보다 자꾸 잊어버리다 그 사람 이다 것',
 '취향 존중 다지 만 진짜 내생 극장 에서 보다 영화 중 가장 노잼 노 감동 임 스토리 어거지 고 감동 어거지',
 'ㄱ 냥 매번 긴장 되다 재밋음',
 '차다 사람 웃기다 바스코 이기 면 락스 코 라고 끄다 바비 이기 면 아이돌 이라고 깔다 그냥 끄다 안달 난 것 처럼 보이다',
 '굿바이 레닌 표절 인 것 이해 왜 뒤 로 갈수록 재미 없어지다',
 '이건 정말 깨알 캐스팅 질퍽 하 지 않다 산뜻하다 내 용구성 자다 버무러진 깨알 일드',
 '약탈 위 변명 이르다 저 놈 착하다 놈 절대 아니다 걸 요',
 '나름 심오하다 뜻 있다 듯 그냥 학생 선생 놀다 영화 절대 아니다',
 '보다 웃다 않다 건 불가능하다',
 '재미없다 지루하다 같다 음식 영화 인데 바베트 만찬 하고 넘다 차이나다 바베트 만찬 이야기 있다 음식 보다 

### 4. Feature Engineering + Model Training / Evaluation

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [29]:
cvect = CountVectorizer()
lr = LogisticRegression(random_state=2022)
pipeline = Pipeline([('CVECT', cvect), ('LR', lr)])
%time pipeline.fit(X_train, y_train)

CPU times: total: 14 s
Wall time: 2.94 s


Pipeline(steps=[('CVECT', CountVectorizer()),
                ('LR', LogisticRegression(random_state=2022))])

In [30]:
pipeline.score(X_test, y_test)

0.8272742159993449

### 5. Test

In [32]:
review1 = '모든 국민이 봤으면 하는 영화입니다.'
review2 = '생각보다 지루하고 별로였네요... 보면서 좀 졸았습니다.'

In [33]:
# 전처리
import re
review1 = re.sub('[^가-힣]',' ', review1)
review2 = re.sub('[^가-힣]',' ', review2)

In [34]:
morphs = okt.morphs(review1, stem=True)
review1 = ' '.join([word for word in morphs if word not in stopwords])
morphs = okt.morphs(review2, stem=True)
review2 = ' '.join([word for word in morphs if word not in stopwords])

In [35]:
pipeline.predict([review1, review2])

array([1, 0], dtype=int64)

### 6. Parameter Search

In [36]:
from sklearn.model_selection import GridSearchCV
params = {
    'CVECT__ngram_range': [(1,1),(1,2)],
    'CVECT__max_df': [0.95, 0.98],
    'LR__C': [1, 5]
}

In [37]:
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3, n_jobs=-1)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 55.1 s
Wall time: 56.4 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT', CountVectorizer()),
                                       ('LR',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'CVECT__max_df': [0.95, 0.98],
                         'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LR__C': [1, 5]},
             scoring='accuracy')

In [38]:
grid_pipe.best_params_

{'CVECT__max_df': 0.95, 'CVECT__ngram_range': (1, 2), 'LR__C': 1}

In [39]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.8480307868664538